In [1]:
import pandas as pd
#from keras.preprocessing import image
from matplotlib import pyplot
from matplotlib.pyplot import imshow
import split_data as sd
import basic_model as n
import numpy as np
from sklearn.metrics import confusion_matrix
#import Network as net
import vectorised as v
from random import randint
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
config=tf.ConfigProto()
config.gpu_options.allow_growth=True

Using TensorFlow backend.


In [2]:
data=pd.read_csv('/home/snehao/dog_breed/Image_Classification/labels.csv')
print(data.head())
targets_series = pd.Series(data['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)
one_hot_labels = np.asarray(one_hot)

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


In [3]:
x_train,x_test,y_train,y_test=sd.split_data(data,one_hot_labels,.10)
#y_test1=x_test['breed'].values
#print(x_train.nlargest(5,x_train.groupby(['breed']).size()))
#x_val,x_test,y_val,y_test=sd.split_data(x_test,y_test,.50)
#print(x_train.groupby(['breed']).size().sort_values())
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9199, 2)
(9199, 120)
(1023, 2)
(1023, 120)


In [4]:
x=[]
y=[]
#for i in range(100):
for i in range(x_train.shape[0]):
    img = image.load_img('/home/snehao/dog_breed/train/'+x_train.iloc[i][0]+'.jpg',target_size=(300,300,3))
    arr=image.img_to_array(img)
    x.append(arr/255)
    y.append(y_train[i])
x=np.array(x)
y=np.array(y)
print('x_shape',x.shape)
print('y_shape',y.shape)

x_shape (9199, 300, 300, 3)
y_shape (9199, 120)


In [ ]:
np.save("xceptiontrainx.npy",x)
np.save("xceptiontrainy.npy",y)

In [5]:
import cv2
x_val_images=[]
y_val1=[]
#for i in range(100):
for i in range(x_test.shape[0]): 
    
    img = image.load_img('/home/snehao/dog_breed/train/'+x_test.iloc[i][0]+'.jpg',target_size=(300,300,3))
    arr=image.img_to_array(img)
    x_val_images.append(arr/255)
    y_val1.append(y_test[i])
x_val_images=np.array(x_val_images)
y_val1=np.array(y_val1)
print('y_shape',y_val1.shape)
print('x_shape',x_val_images.shape)


y_shape (1023, 120)
x_shape (1023, 300, 300, 3)


In [ ]:
np.save("xceptionvalx.npy",x_val_images)
np.save("xceptionvaly.npy",y_val1)

In [6]:
from keras.layers import Concatenate,Add,BatchNormalization,Input,Dropout,Dense,Activation,Flatten,MaxPooling2D,Conv2D,ZeroPadding2D
from keras.models import Model,Sequential
from keras.applications.xception import Xception
base_model = Xception(input_shape=(300,300,3),weights='imagenet', include_top=False,pooling='avg')

In [7]:
callback=keras.callbacks.TensorBoard(log_dir='Graphs_x', histogram_freq=0,  
          write_graph=True, write_images=True)

In [8]:
featuresx=base_model.predict(x)

In [9]:
features_val=base_model.predict(x_val_images)


In [10]:
featuresx.shape

(9199, 2048)

In [ ]:
#featuresxf=base_model.predict(np.flip(x,axis=2))
#features_valf=base_model.predict(np.flip(x_val_images,axis=2))

In [22]:
from keras import optimizers
input_shape=Input((2048,)) 
x1=Dense(1024, activation='relu')(input_shape) 
#x1=Dropout(0.4)(x1) 
#x1=Dense(512, activation='relu')(x1) 
#x1=Dropout(0.2)(x1) 
x1=Dense(120, activation='softmax')(x1) 
model=Model(inputs=input_shape,outputs=x1) 
print(model.summary()) 




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_6 (Dense)              (None, 120)               123000    
Total params: 2,221,176
Trainable params: 2,221,176
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
sgd=optimizers.SGD(momentum=0.0, decay=0.0, nesterov=False) 
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [24]:
model.fit(featuresx,y,epochs=250,validation_data=(features_val,y_val1),batch_size=150,callbacks=[callback])

Train on 9199 samples, validate on 1023 samples
Epoch 1/250
9199/9199 [==============================] - 0s 32us/step - loss: 4.7157 - acc: 0.0377 - val_loss: 4.6338 - val_acc: 0.0890
Epoch 2/250
9199/9199 [==============================] - 0s 24us/step - loss: 4.5551 - acc: 0.1427 - val_loss: 4.4768 - val_acc: 0.2287
Epoch 3/250
9199/9199 [==============================] - 0s 24us/step - loss: 4.3906 - acc: 0.3197 - val_loss: 4.3127 - val_acc: 0.4047
Epoch 4/250
9199/9199 [==============================] - 0s 24us/step - loss: 4.2168 - acc: 0.4778 - val_loss: 4.1365 - val_acc: 0.5239
Epoch 5/250
9199/9199 [==============================] - 0s 24us/step - loss: 4.0306 - acc: 0.5810 - val_loss: 3.9478 - val_acc: 0.6002
Epoch 6/250
9199/9199 [==============================] - 0s 25us/step - loss: 3.8310 - acc: 0.6514 - val_loss: 3.7441 - val_acc: 0.6549
Epoch 7/250
9199/9199 [==============================] - 0s 25us/step - loss: 3.6180 - acc: 0.6978 - val_loss: 3.5281 - val_acc: 0.6911


9199/9199 [==============================] - 0s 25us/step - loss: 0.3846 - acc: 0.9135 - val_loss: 0.4268 - val_acc: 0.8847
Epoch 61/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.3810 - acc: 0.9133 - val_loss: 0.4240 - val_acc: 0.8847
Epoch 62/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.3776 - acc: 0.9128 - val_loss: 0.4210 - val_acc: 0.8847
Epoch 63/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.3742 - acc: 0.9140 - val_loss: 0.4184 - val_acc: 0.8817
Epoch 64/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.3710 - acc: 0.9128 - val_loss: 0.4165 - val_acc: 0.8856
Epoch 65/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.3679 - acc: 0.9143 - val_loss: 0.4140 - val_acc: 0.8837
Epoch 66/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.3650 - acc: 0.9146 - val_loss: 0.4110 - val_acc: 0.8847
Epoch 67/250
9199/9199 [==============================

Epoch 120/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2729 - acc: 0.9280 - val_loss: 0.3502 - val_acc: 0.8886
Epoch 121/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.2718 - acc: 0.9288 - val_loss: 0.3495 - val_acc: 0.8886
Epoch 122/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2706 - acc: 0.9283 - val_loss: 0.3502 - val_acc: 0.8895
Epoch 123/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2697 - acc: 0.9288 - val_loss: 0.3492 - val_acc: 0.8876
Epoch 124/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.2687 - acc: 0.9293 - val_loss: 0.3485 - val_acc: 0.8895
Epoch 125/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2676 - acc: 0.9298 - val_loss: 0.3483 - val_acc: 0.8876
Epoch 126/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2667 - acc: 0.9289 - val_loss: 0.3471 - val_acc: 0.8847
Epoch 127/250
9199/9199 [=========

9199/9199 [==============================] - 0s 24us/step - loss: 0.2249 - acc: 0.9398 - val_loss: 0.3318 - val_acc: 0.8876
Epoch 180/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2243 - acc: 0.9398 - val_loss: 0.3303 - val_acc: 0.8886
Epoch 181/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2235 - acc: 0.9398 - val_loss: 0.3308 - val_acc: 0.8886
Epoch 182/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.2230 - acc: 0.9402 - val_loss: 0.3307 - val_acc: 0.8886
Epoch 183/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2223 - acc: 0.9400 - val_loss: 0.3299 - val_acc: 0.8876
Epoch 184/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2217 - acc: 0.9399 - val_loss: 0.3299 - val_acc: 0.8905
Epoch 185/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.2212 - acc: 0.9404 - val_loss: 0.3298 - val_acc: 0.8895
Epoch 186/250
9199/9199 [=======================

9199/9199 [==============================] - 0s 23us/step - loss: 0.1931 - acc: 0.9483 - val_loss: 0.3246 - val_acc: 0.8905
Epoch 239/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.1926 - acc: 0.9479 - val_loss: 0.3238 - val_acc: 0.8925
Epoch 240/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.1921 - acc: 0.9486 - val_loss: 0.3245 - val_acc: 0.8915
Epoch 241/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.1917 - acc: 0.9492 - val_loss: 0.3233 - val_acc: 0.8915
Epoch 242/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.1912 - acc: 0.9501 - val_loss: 0.3234 - val_acc: 0.8915
Epoch 243/250
9199/9199 [==============================] - 0s 24us/step - loss: 0.1908 - acc: 0.9490 - val_loss: 0.3230 - val_acc: 0.8925
Epoch 244/250
9199/9199 [==============================] - 0s 23us/step - loss: 0.1903 - acc: 0.9490 - val_loss: 0.3234 - val_acc: 0.8915
Epoch 245/250
9199/9199 [=======================

In [ ]:
model.save_weights('xceptiono.h5')


In [ ]:
model.fit(featuresxf,y,epochs=100,validation_data=(features_valf,y_val1),batch_size=150)

In [ ]:
model.save_weights('xceptionf.h5')


In [ ]:
test="/home/snehao/dog_breed"

In [ ]:
files=!ls {"/home/snehao/dog_breed/Image_Classification/"}test
i=0
x_predict=[]
for i in range(len(files)):
    img1=image.load_img(f'{"/home/snehao/dog_breed/Image_Classification/"}test/{files[i]}',target_size=(300,300,3))
    arr=image.img_to_array(img1)
    x_predict.append(arr/255)
x_predict=np.array(x_predict)

In [ ]:
features_test=base_model.predict(x_predict)

In [ ]:
features_testf=base_model.predict(np.flip(x_predict,axis=2))

In [ ]:

predics=model.predict(features_test)
predicsf=model.predict(features_testf)
final=(predics+predicsf)/2
np.savetxt('xception_resultsf.csv',predics,fmt='%f',delimiter=',')

